<a href="https://colab.research.google.com/github/kamaleshpantra/StockPulse/blob/main/Stockpulse_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-flink

In [ ]:
import subprocess
import sys

def install_packages():
    packages = [
        "pyflink",
        "praw==7.8.1",
        "yfinance==0.2.54",
        "streamlit==1.43.2",
        "pyngrok==7.2.3",
        "nltk==3.9.1",
        "pandas==2.2.2",
        "matplotlib==3.10.0",
        "plotly==5.24.1",
        "scikit-learn==1.6.1",
        "tensorflow==2.18.0"
    ]
    for pkg in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
            print(f"Installed {pkg}")
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {pkg}: {e}")
            raise

install_packages()
print("All dependencies installed successfully!")

Installed pyflink
Installed praw==7.8.1
Installed yfinance==0.2.54
Installed streamlit==1.43.2
Installed pyngrok==7.2.3
Installed nltk==3.9.1
Installed pandas==2.2.2
Installed matplotlib==3.10.0
Installed plotly==5.24.1
Installed scikit-learn==1.6.1
Installed tensorflow==2.18.0
All dependencies installed successfully!


In [ ]:
import os
import logging
import nltk
import praw
import yfinance as yf
from pyflink.table import EnvironmentSettings, TableEnvironment, StreamTableEnvironment
from pyflink.table.udf import udf
from pyflink.datastream import StreamExecutionEnvironment
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import json
from datetime import datetime, timedelta
import time
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go
import streamlit as st
from pyngrok import ngrok
from google.colab import userdata

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    filename="/content/stock_predict.log",
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Download NLTK data
nltk.download('vader_lexicon', quiet=True)

# Define sector
sectors = {"Technology": ["AAPL", "MSFT", "GOOGL", "TSLA"]}

# Initialize Reddit API with Colab Secrets
try:
    client_id = userdata.get("REDDIT_CLIENT_ID")
    client_secret = userdata.get("REDDIT_CLIENT_SECRET")
    user_agent = userdata.get("REDDIT_USER_AGENT") or "StockSentiment/1.0"
    if not all([client_id, client_secret, user_agent]):
        raise ValueError("One or more Reddit credentials are missing")
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent,
        read_only=True,
        requestor_kwargs={"timeout": 10}
    )
    limits = reddit.auth.limits
    logger.info("Reddit API initialized. Rate limit remaining: %s", limits.get("remaining", "unknown"))
    print("Reddit API initialized")
except Exception as e:
    logger.error(f"Failed to initialize Reddit API: {e}")
    raise ValueError(
        "Reddit API setup failed. Ensure REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET, "
        "and REDDIT_USER_AGENT are set in Colab Secrets."
    )

# Initialize VADER
sid = SentimentIntensityAnalyzer()

# Define date range
end_date = datetime.now().date()
start_date = end_date - timedelta(days=365)

# Helper for trading days
def next_trading_day(date):
    """Map to next trading day (skip weekends)."""
    date = pd.to_datetime(date)
    while date.weekday() >= 5:  # Saturday or Sunday
        date += timedelta(days=1)
    return date.date()

logger.info("Setup complete")
print("Setup complete!")
print(start_date,end_date)

Reddit API initialized
Setup complete!
2024-04-19 2025-04-19


In [ ]:
import os
import logging
import json
import time
from datetime import datetime, timedelta

def fetch_large_reddit(sector, subreddits=["wallstreetbets", "stocks", "investing"], limit_per_query=500):
    """
    Fetch Reddit posts for sector companies within date range, ensuring relevance.

    Args:
        sector (str): Sector name (e.g., 'Technology').
        subreddits (list): Subreddits to search.
        limit_per_query (int): Max posts per subreddit.

    Returns:
        str: Path to JSONL file.
    """
    logger.info(f"Fetching Reddit data for {sector}")
    filename = f"/content/reddit_{sector}_large.jsonl"

    if os.path.exists(filename):
        logger.info(f"Using cached Reddit data: {filename}")
        print(f"Using cached Reddit data: {filename}")
        return filename

    companies = sectors[sector]
    all_posts = []
    start_timestamp = datetime.combine(start_date, datetime.min.time()).timestamp()
    end_timestamp = datetime.combine(end_date + timedelta(days=1), datetime.min.time()).timestamp() - 1

    for company in companies:
        for subreddit in subreddits:
            for attempt in range(3):
                try:
                    logger.info(f"Attempt {attempt+1}: Fetching {company} from r/{subreddit}")
                    print(f"Fetching {company} from r/{subreddit} (Attempt {attempt+1})...")
                    submissions = reddit.subreddit(subreddit).search(
                        query=company, sort="new", limit=limit_per_query, time_filter="year"
                    )
                    post_count = 0
                    for submission in submissions:
                        if start_timestamp <= submission.created_utc <= end_timestamp:
                            text = submission.title + " " + (submission.selftext or "")
                            if company.lower() in text.lower():
                                post = {
                                    "company": company,
                                    "text": text,
                                    "timestamp": submission.created_utc,
                                    "subreddit": subreddit
                                }
                                all_posts.append(post)
                                post_count += 1
                    logger.info(f"Fetched {post_count} posts for {company} in r/{subreddit}")
                    print(f"Fetched {post_count} posts for {company} in r/{subreddit}")
                    break  # Success, move to next subreddit
                except Exception as e:
                    logger.error(f"Attempt {attempt+1} failed for {company} in {subreddit}: {e}")
                    print(f"Error for {company} in {subreddit}: {e}")
                    if attempt < 2:
                        time.sleep(5)  # Wait before retry
                    continue
                finally:
                    time.sleep(2)  # Rate limit delay

    if not all_posts:
        logger.warning(f"No Reddit posts fetched for {sector}")
        print(f"No Reddit posts fetched for {sector}")
        return filename

    with open(filename, "w") as f:
        for post in all_posts:
            f.write(json.dumps(post) + "\n")

    logger.info(f"Saved {len(all_posts)} posts to {filename}")
    print(f"Saved {len(all_posts)} posts to {filename}")
    return filename

sector = "Technology"
reddit_file = fetch_large_reddit(sector)

Using cached Reddit data: /content/reddit_Technology_large.jsonl


In [ ]:
import os
import logging
import yfinance as yf

def fetch_stock_data(sector, cache_dir="/content/data"):
    """Fetch stock prices for sector companies within date range."""
    logger.info(f"Fetching stock data for {sector}")
    os.makedirs(cache_dir, exist_ok=True)
    companies = sectors[sector]

    for company in companies:
        cache_file = f"{cache_dir}/stock_{company}.csv"
        if os.path.exists(cache_file):
            logger.info(f"Using cached stock data for {company}")
            print(f"Using cached stock data for {company}")
            continue
        try:
            print(f"Fetching stock data for {company}...")
            stock = yf.Ticker(company)
            hist = stock.history(start=start_date, end=end_date + timedelta(days=1), interval="1d")
            if hist.empty:
                logger.warning(f"No stock data for {company}")
                print(f"No stock data for {company}")
                continue
            hist = hist.reset_index()[["Date", "Open", "Close", "High", "Low", "Volume"]]
            hist["Date"] = pd.to_datetime(hist["Date"]).dt.date
            hist.to_csv(cache_file, index=False)
            logger.info(f"Saved {len(hist)} days for {company} to {cache_file}")
            print(f"Saved {len(hist)} days for {company}")
        except Exception as e:
            logger.error(f"Error fetching stock data for {company}: {e}")
            print(f"Error for {company}: {e}")
            continue

fetch_stock_data(sector="Technology")

Using cached stock data for AAPL
Using cached stock data for MSFT
Using cached stock data for GOOGL
Using cached stock data for TSLA


In [ ]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.udf import udf
import pandas as pd
import logging
import os
import json

def preprocess_reddit(sector):
    """Process Reddit posts with Flink to compute daily sentiment in batch mode."""
    logger.info(f"Preprocessing Reddit data for {sector}")
    env_settings = EnvironmentSettings.in_batch_mode()
    t_env = TableEnvironment.create(env_settings)

    @udf(result_type="MAP<STRING, STRING>")
    def parse_json(line):
        """Parse JSON string into a map."""
        try:
            data = json.loads(line)
            return {k: str(v) for k, v in data.items()}
        except:
            return {}

    @udf(result_type="FLOAT")
    def get_vader_sentiment(text):
        try:
            return sid.polarity_scores(text or "")["compound"]
        except:
            return 0.0

    @udf(result_type="STRING")
    def to_trading_date(timestamp):
        try:
            date = pd.to_datetime(float(timestamp), unit="s")
            return str(next_trading_day(date))
        except:
            return str(end_date)

    t_env.create_temporary_function("parse_json", parse_json)
    t_env.create_temporary_function("get_vader_sentiment", get_vader_sentiment)
    t_env.create_temporary_function("to_trading_date", to_trading_date)

    reddit_file = f"/content/reddit_{sector}_large.jsonl"
    if not os.path.exists(reddit_file):
        logger.warning(f"No Reddit data for {sector}")
        print(f"No Reddit data for {sector}")
        return

    t_env.execute_sql("""
        CREATE TABLE reddit_source (
            line STRING
        ) WITH (
            'connector' = 'filesystem',
            'path' = 'file://%s',
            'format' = 'raw'
        )
    """ % reddit_file)

    t_env.execute_sql("""
        CREATE TEMPORARY VIEW parsed_reddit AS
        SELECT
            parsed['company'] AS company,
            to_trading_date(parsed['timestamp']) AS trading_date,
            get_vader_sentiment(parsed['text']) AS sentiment_score
        FROM (
            SELECT parse_json(line) AS parsed
            FROM reddit_source
            WHERE line IS NOT NULL
        ) t
        WHERE parsed['company'] IS NOT NULL AND parsed['timestamp'] IS NOT NULL
    """)

    agg_table = t_env.sql_query("""
        SELECT
            company,
            trading_date,
            AVG(sentiment_score) AS avg_sentiment,
            COUNT(*) AS post_count
        FROM parsed_reddit
        GROUP BY company, trading_date
    """)

    sink_file = f"/content/daily_sentiment_{sector}.csv"
    t_env.execute_sql("""
        CREATE TABLE sentiment_sink (
            company STRING,
            trading_date STRING,
            avg_sentiment FLOAT,
            post_count BIGINT
        ) WITH (
            'connector' = 'filesystem',
            'path' = 'file://%s',
            'format' = 'csv'
        )
    """ % sink_file)

    agg_table.execute_insert("sentiment_sink").wait()
    logger.info(f"Processed Reddit posts to {sink_file}")
    print(f"Processed Reddit posts for {sector}")

def preprocess_stock(sector):
    """Process stock data with Flink for trends."""
    logger.info(f"Preprocessing stock data for {sector}")
    env_settings = EnvironmentSettings.in_batch_mode()
    t_env = TableEnvironment.create(env_settings)

    companies = sectors[sector]
    for company in companies:
        cache_file = f"/content/data/stock_{company}.csv"
        if not os.path.exists(cache_file):
            logger.warning(f"Stock data missing for {company}")
            print(f"Stock data missing for {company}")
            continue

        source_table = f"stock_source_{company}"
        t_env.execute_sql("""
            CREATE TABLE %s (
                `Date` STRING,
                `Open` DOUBLE,
                `Close` DOUBLE,
                `High` DOUBLE,
                `Low` DOUBLE,
                `Volume` BIGINT
            ) WITH (
                'connector' = 'filesystem',
                'path' = 'file://%s',
                'format' = 'csv',
                'csv.ignore-parse-errors' = 'true'
            )
        """ % (source_table, cache_file))

        table = t_env.sql_query("""
            SELECT
                `Date` AS `date`,
                `Close` AS `close`,
                CASE
                    WHEN LEAD(`Close`) OVER (ORDER BY `Date`) > `Close` THEN 1
                    ELSE 0
                END AS `trend`
            FROM %s
            WHERE `Date` IS NOT NULL
        """ % source_table)

        sink_file = f"/content/processed_stock_{company}.csv"
        sink_table = f"stock_sink_{company}"
        t_env.execute_sql("""
            CREATE TABLE %s (
                `date` STRING,
                `close` DOUBLE,
                `trend` INT
            ) WITH (
                'connector' = 'filesystem',
                'path' = 'file://%s',
                'format' = 'csv'
            )
        """ % (sink_table, sink_file))

        table.execute_insert(sink_table).wait()
        logger.info(f"Processed stock data to {sink_file}")
        print(f"Processed stock data for {company}")

preprocess_reddit(sector="Technology")
preprocess_stock(sector="Technology")

Processed Reddit posts for Technology
Processed stock data for AAPL
Processed stock data for MSFT
Processed stock data for GOOGL
Processed stock data for TSLA


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging
import os
import glob

# Assuming logger, sectors from Segment 2
logger.info("Starting LSTM data preparation")

def prepare_lstm_data(sector, sequence_length=60):
    """Prepare sequences for LSTM from sentiment and stock data."""
    logger.info(f"Preparing LSTM data for {sector}")

    # Find sentiment part file
    sentiment_dir = f"/content/daily_sentiment_{sector}.csv"
    sentiment_files = glob.glob(f"{sentiment_dir}/part-*")
    if not sentiment_files:
        logger.error(f"No part files found in {sentiment_dir}")
        print(f"No part files found in {sentiment_dir}")
        return []

    sentiment_part_file = sentiment_files[0]  # Take first part file
    if not os.path.isfile(sentiment_part_file):
        logger.error(f"Sentiment part file is not a file: {sentiment_part_file}")
        print(f"Sentiment part file is not a file: {sentiment_part_file}")
        return []

    # Read sentiment data with explicit column names
    try:
        sentiment_df = pd.read_csv(
            sentiment_part_file,
            names=['company', 'trading_date', 'avg_sentiment', 'post_count'],
            header=None
        )
        logger.info(f"Loaded sentiment data: {len(sentiment_df)} rows")
        print(f"Loaded sentiment data: {len(sentiment_df)} rows")
        print(f"Sentiment columns: {list(sentiment_df.columns)}")
        print("Sentiment sample:")
        print(sentiment_df.head().to_string())
        unique_companies = sentiment_df['company'].unique()
        print(f"Unique companies: {unique_companies}")
    except Exception as e:
        logger.error(f"Failed to read sentiment part file {sentiment_part_file}: {e}")
        print(f"Failed to read sentiment part file: {e}")
        return []

    # Validate sentiment columns
    required_cols = ['company', 'trading_date', 'avg_sentiment', 'post_count']
    missing_cols = [col for col in required_cols if col not in sentiment_df.columns]
    if missing_cols:
        logger.error(f"Missing required columns in sentiment data: {missing_cols}")
        print(f"Missing required columns in sentiment data: {missing_cols}")
        return []

    companies = sectors[sector]
    sequences = []
    scaler = MinMaxScaler()

    for company in companies:
        # Find stock part file
        stock_dir = f"/content/processed_stock_{company}.csv"
        stock_files = glob.glob(f"{stock_dir}/part-*")
        if not stock_files:
            logger.warning(f"No part files found in {stock_dir}")
            print(f"No part files found for {company}")
            continue

        stock_part_file = stock_files[0]  # Take first part file
        if not os.path.isfile(stock_part_file):
            logger.warning(f"Stock part file is not a file: {stock_part_file}")
            print(f"Stock part file is not a file for {company}")
            continue

        # Read stock data with explicit column names
        try:
            stock_df = pd.read_csv(
                stock_part_file,
                names=['date', 'close', 'trend'],
                header=None
            )
            logger.info(f"Loaded stock data for {company}: {len(stock_df)} rows")
            print(f"Loaded stock data for {company}: {len(stock_df)} rows")
            print(f"Stock columns for {company}: {list(stock_df.columns)}")
            print(f"Stock sample for {company}:")
            print(stock_df.head().to_string())
        except Exception as e:
            logger.error(f"Failed to read stock part file {stock_part_file}: {e}")
            print(f"Failed to read stock part file for {company}: {e}")
            continue

        # Validate stock columns
        required_stock_cols = ['date', 'close', 'trend']
        missing_stock_cols = [col for col in required_stock_cols if col not in stock_df.columns]
        if missing_stock_cols:
            logger.error(f"Missing required columns in stock data for {company}: {missing_stock_cols}")
            print(f"Missing required columns in stock data for {company}: {missing_stock_cols}")
            continue

        # Merge data
        try:
            company_sentiment = sentiment_df[sentiment_df['company'] == company][
                ['trading_date', 'avg_sentiment', 'post_count']
            ]
            merged_df = stock_df.merge(
                company_sentiment,
                left_on='date',
                right_on='trading_date',
                how='left'
            )
            merged_df['avg_sentiment'] = merged_df['avg_sentiment'].fillna(0.0)
            merged_df['post_count'] = merged_df['post_count'].fillna(0)
            merged_df = merged_df.drop(columns=['trading_date'], errors='ignore')
            logger.info(f"Merged data for {company}: {len(merged_df)} rows")
            print(f"Merged data for {company}: {len(merged_df)} rows")
        except Exception as e:
            logger.error(f"Failed to merge data for {company}: {e}")
            print(f"Failed to merge data for {company}: {e}")
            continue

        # Prepare features and target
        try:
            features = merged_df[['close', 'avg_sentiment', 'post_count']].values
            target = merged_df['trend'].values

            if len(features) > 0:
                scaled_features = scaler.fit_transform(features)
            else:
                logger.warning(f"No features for {company}")
                print(f"No features for {company}")
                continue

            X, y = [], []
            for i in range(len(scaled_features) - sequence_length):
                X.append(scaled_features[i:i + sequence_length])
                y.append(target[i + sequence_length])

            if X:
                sequences.append((company, np.array(X), np.array(y)))
                logger.info(f"Created {len(X)} sequences for {company}")
                print(f"Created {len(X)} sequences for {company}")
            else:
                logger.warning(f"No sequences for {company}")
                print(f"No sequences for {company}")
        except Exception as e:
            logger.error(f"Failed to create sequences for {company}: {e}")
            print(f"Failed to create sequences for {company}: {e}")

    total_sequences = sum(len(X) for _, X, _ in sequences)
    logger.info(f"Total sequences prepared: {total_sequences}")
    print(f"Total sequences prepared: {total_sequences}")
    return sequences

# Run for Technology sector
lstm_data = prepare_lstm_data("Technology")

Loaded sentiment data: 716 rows
Sentiment columns: ['company', 'trading_date', 'avg_sentiment', 'post_count']
Sentiment sample:
  company trading_date  avg_sentiment  post_count
0    AAPL   2025-04-17       0.204750           2
1    AAPL   2025-04-09       0.106567           6
2    AAPL   2025-04-03       0.450271           7
3    AAPL   2025-04-01       0.593433           3
4    AAPL   2025-03-20       0.036550           2
Unique companies: ['AAPL' 'MSFT' 'GOOGL' 'TSLA']
Loaded stock data for AAPL: 251 rows
Stock columns for AAPL: ['date', 'close', 'trend']
Stock sample for AAPL:
         date       close  trend
0  2024-04-19  164.224564      1
1  2024-04-22  165.060593      1
2  2024-04-23  166.115616      1
3  2024-04-24  168.225662      1
4  2024-04-25  169.091568      0
Merged data for AAPL: 251 rows
Created 191 sequences for AAPL
Loaded stock data for MSFT: 251 rows
Stock columns for MSFT: ['date', 'close', 'trend']
Stock sample for MSFT:
         date       close  trend
0  2024-

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import logging
import os

# Assuming logger from Segment 2
logger.info("Starting LSTM training")

def train_lstm_models(lstm_data, epochs=100, batch_size=32):
    """Train LSTM models for each company."""
    models = {}

    for company, X, y in lstm_data:
        logger.info(f"Training LSTM for {company}")
        print(f"Training LSTM for {company}")

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        logger.info(f"{company} - Train: {X_train.shape}, Test: {X_test.shape}")
        print(f"{company} - Train: {X_train.shape}, Test: {X_test.shape}")

        # Build LSTM model
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1, activation='sigmoid')
        ])

        # Compile model
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        # Train model
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1
        )

        # Evaluate model
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        logger.info(f"{company} - Test accuracy: {accuracy:.4f}")
        print(f"{company} - Test accuracy: {accuracy:.4f}")

        # Save model
        model_path = f"/content/model_{company}.h5"
        model.save(model_path)
        logger.info(f"Saved model for {company} to {model_path}")
        print(f"Saved model for {company} to {model_path}")

        models[company] = model

    return models

# Train models using Segment 6 output
models = train_lstm_models(lstm_data)

Training LSTM for AAPL
AAPL - Train: (152, 60, 3), Test: (39, 60, 3)
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 211ms/step - accuracy: 0.4766 - loss: 0.6918 - val_accuracy: 0.5385 - val_loss: 0.6915
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.5435 - loss: 0.6911 - val_accuracy: 0.5385 - val_loss: 0.6917
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.5405 - loss: 0.6890 - val_accuracy: 0.5385 - val_loss: 0.6917
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.5301 - loss: 0.6927 - val_accuracy: 0.5385 - val_loss: 0.6915
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5192 - loss: 0.6976 - val_accuracy: 0.5385 - val_loss: 0.6915
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5865 - loss: 0.6844 - val_accuracy: 0.5385 - val_loss: 0.6916
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.5192 - loss: 0.6927 - val_accuracy: 0.5385 - val_loss: 0.6916
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.5487 - loss: 0.6900 - val_accuracy: 0.5385 - val_loss: 0.6914
E

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from datetime import datetime, timedelta

# Assuming logger, sectors from Segment 2
logger.info("Starting trend predictions")

def predict_trends(lstm_data, models, forecast_days=10):
    """Predict stock trends using trained LSTM models."""
    predictions = []

    for company, X, _ in lstm_data:
        logger.info(f"Predicting trends for {company}")
        print(f"Predicting trends for {company}")

        if company not in models:
            logger.warning(f"No model found for {company}")
            print(f"No model found for {company}")
            continue

        model = models[company]
        # Use latest 60-day sequence for prediction
        latest_sequence = X[-1:]  # Shape: [1, 60, 3]

        # Predict for next forecast_days
        company_predictions = []
        current_sequence = latest_sequence.copy()
        last_date = datetime.strptime("2025-04-16", "%Y-%m-%d")  # Latest stock date

        for day in range(forecast_days):
            # Predict probability
            pred_prob = model.predict(current_sequence, verbose=0)[0][0]
            trend = 1 if pred_prob >= 0.5 else 0  # Threshold at 0.5
            pred_date = last_date + timedelta(days=day + 1)

            company_predictions.append({
                'company': company,
                'date': pred_date.strftime("%Y-%m-%d"),
                'trend': trend,
                'probability': float(pred_prob)
            })

            # Update sequence (shift and append dummy data for demo)
            # In practice, use actual new data if available
            new_data = current_sequence[:, -1, :].copy()
            new_data[:, 0] = new_data[:, 0]  # Keep last close (dummy)
            new_data[:, 1] = 0.0  # Neutral sentiment
            new_data[:, 2] = 0  # No posts
            current_sequence = np.append(current_sequence[:, 1:, :], [new_data], axis=1)

        predictions.extend(company_predictions)
        logger.info(f"Predicted {len(company_predictions)} trends for {company}")
        print(f"Predicted {len(company_predictions)} trends for {company}")

    # Save predictions
    predictions_df = pd.DataFrame(predictions)
    output_path = "/content/predictions.csv"
    predictions_df.to_csv(output_path, index=False)
    logger.info(f"Saved predictions to {output_path}")
    print(f"Saved predictions to {output_path}")

    return predictions_df

# Load models
models = {}
for company in ['AAPL', 'MSFT', 'GOOGL', 'TSLA']:
    model_path = f"/content/model_{company}.h5"
    if os.path.exists(model_path):
        models[company] = tf.keras.models.load_model(model_path)
        print(f"Loaded model for {company}")
    else:
        print(f"Model not found for {company}")

# Predict trends using Segment 6's lstm_data
predictions_df = predict_trends(lstm_data, models)

Loaded model for AAPL
Loaded model for MSFT
Loaded model for GOOGL
Loaded model for TSLA
Predicting trends for AAPL
Predicted 10 trends for AAPL
Predicting trends for MSFT
Predicted 10 trends for MSFT
Predicting trends for GOOGL
Predicted 10 trends for GOOGL
Predicting trends for TSLA
Predicted 10 trends for TSLA
Saved predictions to /content/predictions.csv


In [ ]:
import pandas as pd
import logging

# Assuming logger from Segment 2
logger.info("Starting alert generation")

def generate_alerts(predictions_file='/content/predictions.csv'):
    try:
        predictions = pd.read_csv(predictions_file)
        logger.info(f"Loaded predictions: {len(predictions)} rows")
        print(f"Loaded predictions: {len(predictions)} rows")

        alerts = []
        for _, row in predictions.iterrows():
            probability = row['probability']
            trend = row['trend']
            if probability >= 0.7 and trend == 1:
                alert_type = 'Strong Buy'
                alerts.append({
                    'company': row['company'],
                    'date': row['date'],
                    'alert_type': alert_type,
                    'probability': probability
                })
            elif probability <= 0.3 and trend == 0:
                alert_type = 'Strong Sell'
                alerts.append({
                    'company': row['company'],
                    'date': row['date'],
                    'alert_type': alert_type,
                    'probability': probability
                })

        alerts_df = pd.DataFrame(alerts)
        alerts_df.to_csv('/content/alerts.csv', index=False)
        logger.info(f"Generated {len(alerts)} alerts, saved to /content/alerts.csv")
        print(f"Generated {len(alerts)} alerts, saved to /content/alerts.csv")
    except Exception as e:
        logger.error(f"Alert generation failed: {e}")
        print(f"Alert generation failed: {e}")

generate_alerts()

Loaded predictions: 40 rows
Generated 30 alerts, saved to /content/alerts.csv


In [ ]:
!pip install streamlit pyngrok plotly

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2tXLBQHCEqsQS0TYDRNHF5w8fHk_2rbtDMMHkdDsLPdss1hUP")

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import logging
import os

# Configure Streamlit page
st.set_page_config(
    page_title="Stock Sentiment Dashboard",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Logger setup
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Define CSV paths
PRED_PATH = os.path.abspath('/content/predictions.csv')
ALERTS_PATH = os.path.abspath('/content/alerts.csv')

@st.cache_data(show_spinner=True)
def load_data():
    """
    Load predictions and alerts. Alerts failures are non-fatal.
    Returns: df (DataFrame or None), alerts (DataFrame or empty)
    """
    # Load predictions (fatal if fails)
    try:
        logger.info(f"Loading predictions from {PRED_PATH}")
        df = pd.read_csv(PRED_PATH, parse_dates=['date'])
        logger.info(f"Loaded {len(df)} prediction rows")
    except Exception as e:
        logger.error(f"Error loading predictions: {e}")
        return None, None

    # Load alerts (non-fatal)
    try:
        logger.info(f"Loading alerts from {ALERTS_PATH}")
        alerts = pd.read_csv(ALERTS_PATH, parse_dates=['date'])
        logger.info(f"Loaded {len(alerts)} alert rows")
    except Exception as e:
        alerts = pd.DataFrame()
        logger.warning(f"Unable to load alerts: {e} -- continuing without alerts")

    return df, alerts

# Write out the dashboard script (dashboard.py)
script_lines = [
    'import streamlit as st',
    'import pandas as pd',
    'import plotly.express as px',
    'import logging',
    'import os',
    '',
    "st.set_page_config(page_title='Stock Sentiment Dashboard', layout='wide')",
    '',
    "logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)",
    "logger = logging.getLogger(__name__)",
    '',
    f"PRED_PATH = r'{PRED_PATH}'",
    f"ALERTS_PATH = r'{ALERTS_PATH}'",
    '',
    "@st.cache_data(show_spinner=True)",
    "def load_data():",
    "    try:",
    "        df = pd.read_csv(PRED_PATH, parse_dates=['date'])",
    "    except Exception as e:",
    "        st.error(f'Error loading predictions: {e}')",
    "        return None, None",
    "    try:",
    "        alerts = pd.read_csv(ALERTS_PATH, parse_dates=['date'])",
    "    except Exception as e:",
    "        alerts = pd.DataFrame()",
    "        st.warning(f'Unable to load alerts: {e} - proceeding without alerts')",
    "    return df, alerts",
    '',
    "# Main app",
    "st.title('📈 Stock Sentiment Dashboard')",
    "df, alerts = load_data()",
    "if df is None:",
    "    st.error('Failed to load predictions. Check file path and format.')",
    "else:",
    "    st.write(f'Loaded predictions: {len(df)} rows')",
    "    if not alerts.empty:",
    "        st.write(f'Loaded alerts: {len(alerts)} rows')",
    "    else:",
    "        st.info('No alerts available or failed to load alerts.')",
    "    # Company selector",
    "    companies = df['company'].unique().tolist()",
    "    selected_company = st.sidebar.selectbox('Select Company', companies)",
    "    subdf = df[df['company'] == selected_company]",
    "    fig = px.line(subdf, x='date', y='probability', title=f'Trend Probability for {selected_company}', labels={'probability':'Probability','date':'Date'})",
    "    st.plotly_chart(fig, use_container_width=True)",
    "    # Alerts display",
    "    if not alerts.empty:",
    "        comp_alerts = alerts[alerts['company'] == selected_company]",
    "        if not comp_alerts.empty:",
    "            st.subheader(f'Alerts for {selected_company}')",
    "            st.dataframe(comp_alerts[['date','alert_type','probability']])",
    "        else:",
    "            st.write('No alerts for this company.')",
]
with open('dashboard.py', 'w') as fh:
    fh.write("\n".join(script_lines))

2025-04-19 07:56:45.240 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-19 07:56:45.244 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [ ]:
from pyngrok import ngrok
import subprocess
import time

# Start Streamlit
streamlit_proc = subprocess.Popen(["streamlit", "run", "/content/dashboard.py", "--server.port", "8501"])
time.sleep(5)  # Wait for Streamlit to start
public_url = ngrok.connect(8501)
print(f"Streamlit dashboard running at: {public_url}")

Streamlit dashboard running at: NgrokTunnel: "https://e3c3-35-239-177-220.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import logging

# Assuming logger from Segment 2
logger.info("Starting local plotting dashboard")

# Set Seaborn style
sns.set_theme(style="darkgrid")
sns.set_palette("deep")

def plot_dashboard(predictions_file="/content/predictions.csv", alerts_file="/content/alerts.csv"):
    """Plot predictions locally using Matplotlib and Seaborn, skip alerts if invalid."""
    # Check if predictions file exists
    if not os.path.exists(predictions_file):
        logger.error(f"Predictions file not found: {predictions_file}")
        print(f"Predictions file not found: {predictions_file}")
        return

    # Load predictions
    try:
        predictions = pd.read_csv(predictions_file)
        if predictions.empty or not all(col in predictions for col in ['company', 'date', 'trend', 'probability']):
            raise ValueError("Predictions CSV is empty or missing required columns")
        logger.info(f"Loaded predictions: {len(predictions)} rows")
        print(f"Loaded predictions: {len(predictions)} rows")
        # Print predictions DataFrame
        print("\nPredictions DataFrame:")
        print(predictions[['company', 'date', 'trend', 'probability']].to_string(index=False))
    except Exception as e:
        logger.error(f"Failed to load predictions: {e}")
        print(f"Failed to load predictions: {e}")
        return

    # Load alerts (allow failure)
    alerts = pd.DataFrame(columns=['company', 'date', 'alert_type', 'probability'])
    try:
        alerts_temp = pd.read_csv(alerts_file)
        if not alerts_temp.empty and all(col in alerts_temp for col in ['company', 'date', 'alert_type', 'probability']):
            alerts = alerts_temp
            logger.info(f"Loaded alerts: {len(alerts)} rows")
            print(f"Loaded alerts: {len(alerts)} rows")
        else:
            logger.warning("Alerts CSV is empty or has invalid columns, skipping alerts plotting")
            print("Alerts CSV is empty or has invalid columns, skipping alerts plotting")
            print("Plotting predictions only")
    except Exception as e:
        logger.warning(f"Failed to load alerts: {e}, skipping alerts plotting")
        print(f"Failed to load alerts: {e}, skipping alerts plotting")
        print("Plotting predictions only")

    # Create output directory
    os.makedirs("/content/plots", exist_ok=True)

    # Plot for each company
    companies = predictions['company'].unique()
    for company in companies:
        # Filter data
        company_predictions = predictions[predictions['company'] == company]

        # Plot 1: Predictions Table
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.axis('off')
        table_data = company_predictions[['date', 'trend', 'probability']].round(3)
        table = ax.table(cellText=table_data.values, colLabels=table_data.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)
        plt.title(f"Predictions for {company}", pad=20)
        plt.savefig(f"/content/plots/{company}_predictions.png", bbox_inches='tight', dpi=300)
        plt.close()
        logger.info(f"Saved predictions table for {company}")
        print(f"Saved predictions table for {company}")

        # Plot 2: Probability Line Plot
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=company_predictions, x='date', y='probability', marker='o')
        plt.title(f"Trend Probability for {company}")
        plt.xlabel("Date")
        plt.ylabel("Trend Probability")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"/content/plots/{company}_probability.png", bbox_inches='tight', dpi=300)
        plt.close()
        logger.info(f"Saved probability plot for {company}")
        print(f"Saved probability plot for {company}")

# Run plotting
plot_dashboard()

Loaded predictions: 40 rows

Predictions DataFrame:
company       date  trend  probability
   AAPL 2025-04-17      0     0.491132
   AAPL 2025-04-18      0     0.490191
   AAPL 2025-04-19      0     0.490738
   AAPL 2025-04-20      0     0.492767
   AAPL 2025-04-21      0     0.496155
   AAPL 2025-04-22      1     0.500915
   AAPL 2025-04-23      1     0.506820
   AAPL 2025-04-24      1     0.513527
   AAPL 2025-04-25      1     0.520711
   AAPL 2025-04-26      1     0.528968
   MSFT 2025-04-17      0     0.074227
   MSFT 2025-04-18      0     0.077494
   MSFT 2025-04-19      0     0.079625
   MSFT 2025-04-20      0     0.083903
   MSFT 2025-04-21      0     0.088665
   MSFT 2025-04-22      0     0.100714
   MSFT 2025-04-23      0     0.107225
   MSFT 2025-04-24      0     0.122982
   MSFT 2025-04-25      0     0.129164
   MSFT 2025-04-26      0     0.138835
  GOOGL 2025-04-17      0     0.250819
  GOOGL 2025-04-18      0     0.251715
  GOOGL 2025-04-19      0     0.253206
  GOOGL 2025

In [ ]:
!ls /content/dashboard.py
!head -n 10 /content/dashboard.py

/content/dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import logging
import os

st.set_page_config(page_title='Stock Sentiment Dashboard', layout='wide')

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
